In [276]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import os
import scipy.stats as stats
import seaborn as sns
import re

# Предварительный просмотр данных

In [277]:
data_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'dataset//housing_market_dataset.csv'))

dataset = pd.read_csv(data_path)
dataset.head(3)

,url,Заголовок,Новостройка,Этап строительства,Год постройки,Класс жилья,МЦК,Кол-во просмотров,Кол-во просмотров сегодня,Описание,...,Ближайшая станция метро,Расстояние до метро,Ближайшая школа,Рейтинг школы,Адрес ближайшей школы,Расстояние до ближайшей школы,Район,Рейтинг района,Номер этажа,Очищенная цена за м²
0,https://move.ru/objects/moskva_ulica_baykalska...,"Продажа 1-комнатной квартиры, 31 м², Москва, у...",NaN,NaN,1967.0,NaN,NaN,125,3.0,ВП-738 СВОБОДНАЯ ПРОДАЖА. ДОМ ПОД СНОС. ЮРИДИЧ...,...,м. Щелковская,1294 м.,ГБОУ школа № 1352,Место в рейтинге №47,"107207,г.Москва, Щелковское ш., д. 77 А",190 м,NaN,NaN,3,169355
1,https://move.ru/objects/prodaetsya_2-komn_kvar...,"Продам 2-комнатную квартиру, 83.2 м², Москва, ...","ЖК City Park («Сити Парк»), м. Выставочная",Отделка,2020.0,Элит класс,МЦК Деловой центр,202,1.0,Квартира премиум класса в ЖК CITY PARK с видам...,...,м. Выставочная,510 м.,ГБОУ школа № 2055,Место в рейтинге №456,"123100, г. Москва, ул. 2-я Черногрязская, дом 7",776 м,Пресненский,14 место из 58 в рейтинге районов,4,268000
2,https://move.ru/objects/prodaetsya_2-komn_kvar...,"Продаю 2-комнатную квартиру, 89.6 м², Москва, ...","ЖК Silver (ЖК «Резиденции кинематографистов»),...",Отделка,2018.0,Бизнес класс,МЦК Ботанический сад,20,1.0,"Продается 2-комн. квартира, площадью 89.6 м2 в...",...,м. Свиблово,"12 мин, 1 км",ГБОУ Гимназия «Свиблово»,Место в рейтинге №28,"129323, г. Москва, ул. Седова, д. 4, корп. 1",619 м,Свиблово,15 место из 58 в рейтинге районов,2,174786


In [278]:
dataset.shape

(13001, 37)

In [279]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13001 entries, 0 to 13000
Data columns (total 37 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            13001 non-null  object 
 1   Заголовок                      13001 non-null  object 
 2   Новостройка                    6080 non-null   object 
 3   Этап строительства             5812 non-null   object 
 4   Год постройки                  7025 non-null   float64
 5   Класс жилья                    5812 non-null   object 
 6   МЦК                            7633 non-null   object 
 7   Кол-во просмотров              13001 non-null  int64  
 8   Кол-во просмотров сегодня      7224 non-null   float64
 9   Описание                       12263 non-null  object 
 10  Информация                     13001 non-null  object 
 11  Адрес                          13001 non-null  object 
 12  Тип здания                     12897 non-null 

# Предобработка данных

In [280]:
dataset.duplicated().sum()

11

In [281]:
dataset.drop_duplicates(inplace=True)
dataset.reset_index(drop=True, inplace=True)

In [282]:
dataset.shape

(12990, 37)

## Выбор признаков для анализа

In [283]:
dataset.columns

Index(['url', 'Заголовок', 'Новостройка', 'Этап строительства',
       'Год постройки', 'Класс жилья', 'МЦК', 'Кол-во просмотров',
       'Кол-во просмотров сегодня', 'Описание', 'Информация', 'Адрес',
       'Тип здания', 'Количество этажей', 'Цена', 'цена за м²',
       'Комиссия агенту', 'Количество комнат', 'Этажность', 'Тип объекта',
       'Тип объявления', 'Общая площадь', 'Дата публикации',
       'Дата  обновления', 'Площадь кухни', 'Возможна ипотека', 'Лифт',
       'Ближайшая станция метро', 'Расстояние до метро', 'Ближайшая школа',
       'Рейтинг школы', 'Адрес ближайшей школы',
       'Расстояние до ближайшей школы', 'Район', 'Рейтинг района',
       'Номер этажа', 'Очищенная цена за м²'],
      dtype='object')

In [284]:
columns = ['url','МЦК', 'Описание', 'Информация', 'цена за м²','Дата публикации','Дата  обновления','Адрес ближайшей школы', 'Район',
           'Рейтинг района', 'Кол-во просмотров сегодня','Новостройка','Площадь кухни','Количество этажей']

In [285]:
dataset.drop(columns=columns, inplace=True)
dataset.shape

(12990, 23)

## Проверка признаков

### Этап строительства

In [286]:
dataset['Этап строительства'].unique()

array([nan, 'Отделка', 'Сдан', 'Возведение стен', 'В проекте', 'Площадка',
       'Заморожено', 'Котлован'], dtype=object)

In [287]:
dataset['Этап строительства'].value_counts()

Этап строительства
Возведение стен    3208
Сдан               1050
Отделка             803
Площадка            541
В проекте           159
Котлован             41
Заморожено            4
Name: count, dtype: int64

In [288]:
dataset['Этап строительства'].isnull().sum()

7184

In [289]:
# замена пропусков
dataset.loc[dataset[dataset['Этап строительства'].isnull()].index, 'Этап строительства'] = 'Неизвестно'

# замороженных мало, исключаем эти записи
dataset = dataset[dataset['Этап строительства']!='Заморожено']

# возможны неизвестные дома, которые явно прошли акт сдачи (условно, до 2000 года включительно)
dataset.loc[dataset[dataset['Год постройки'] <= 2000].index, 'Этап строительства'] = 'Сдан'


In [290]:
# проверка изменений
dataset['Этап строительства'].value_counts()

Этап строительства
Неизвестно         4812
Сдан               3424
Возведение стен    3208
Отделка             803
Площадка            541
В проекте           158
Котлован             40
Name: count, dtype: int64

### Год постройки

In [291]:
# пропуски не обрабатываем (нет смысла портить тип столбца)
dataset['Год постройки'].isna().sum()

5970

In [292]:
dataset['Год постройки'].value_counts()

Год постройки
2019.0    2111
2018.0     795
2021.0     564
2020.0     298
2011.0     150
          ... 
1898.0       1
1914.0       1
1885.0       1
1889.0       1
1945.0       1
Name: count, Length: 114, dtype: int64

In [293]:
# заменим на десятилетия, сократив уникальные значения
dataset['Год постройки'] = dataset['Год постройки'] // 10 * 10

In [294]:
# проверка изменения
dataset['Год постройки'].value_counts()

Год постройки
2010.0    3401
2020.0     862
1960.0     737
1970.0     521
2000.0     400
1980.0     336
1950.0     335
1990.0     258
1930.0      57
1920.0      40
1940.0      30
1910.0      21
1900.0      11
1880.0       5
1890.0       1
1870.0       1
Name: count, dtype: int64

### Класс жилья

In [295]:
dataset['Класс жилья'].unique()

array([nan, 'Элит класс', 'Бизнес класс', 'Эконом класс', 'Комфорт класс'],
      dtype=object)

In [296]:
# замена пропусков
dataset.loc[dataset[dataset['Класс жилья'].isnull()].index, 'Класс жилья'] = 'Неизвестно'

In [297]:
# проверка изменений
dataset['Класс жилья'].value_counts()

Класс жилья
Неизвестно       7184
Бизнес класс     3411
Комфорт класс    1871
Элит класс        345
Эконом класс      175
Name: count, dtype: int64

### Тип здания

In [298]:
dataset['Тип здания'].unique()

array(['Панельное', 'Монолитно-кирпичный', 'Монолитное', 'Монолитный',
       'Кирпичное', nan, 'Блочное', 'Монолитно-кирпичное', 'Сталинское',
       'Панельный', 'Панельно-кирпичное'], dtype=object)

In [299]:
dataset['Тип здания'].value_counts()

Тип здания
Монолитное             5223
Кирпичное              4487
Блочное                1662
Монолитно-кирпичное     703
Панельное               593
Монолитно-кирпичный     156
Сталинское               45
Монолитный               10
Панельный                 2
Панельно-кирпичное        1
Name: count, dtype: int64

Исправим двойственные представления одного и того же признака

In [300]:
# замена пропусков
dataset.loc[dataset[dataset['Тип здания'].isnull()].index, 'Тип здания'] = 'Неизвестно'

# приведение к единообразию записи признаков
dataset.loc[dataset[dataset['Тип здания'] == 'Панельный'].index, 'Тип здания'] = 'Панельное'
dataset.loc[dataset[dataset['Тип здания'] == 'Монолитный'].index, 'Тип здания'] = 'Монолитное'
dataset.loc[dataset[dataset['Тип здания'] == 'Монолитно-кирпичный'].index, 'Тип здания'] = 'Монолитно-кирпичное'

# удалим запись с одним наблюдением
dataset = dataset[dataset['Тип здания'] != 'Панельно-кирпичное']

In [301]:
# проверим изменения
dataset['Тип здания'].value_counts()

Тип здания
Монолитное             5233
Кирпичное              4487
Блочное                1662
Монолитно-кирпичное     859
Панельное               595
Неизвестно              104
Сталинское               45
Name: count, dtype: int64

### Цена

In [302]:
dataset['Цена'].sample(5)

9146      9 000 000 ₽
4154     16 725 300 ₽
1064     38 412 500 ₽
6948      6 100 000 ₽
10228     8 100 000 ₽
Name: Цена, dtype: object

Преобразуем столбец к нормальному численному представлению

In [303]:
# удалим знак ₽
dataset['Цена'] = dataset['Цена'].apply(lambda x: x[:-1] if pd.notna(x) else np.nan)

# заменим лишние пробелы
dataset['Цена'] = dataset['Цена'].str.replace(' ', '')

# приведем получившийся object к числовому типу
dataset['Цена'] = dataset['Цена'].astype('int64')

# опишем цену в млн. дабы в датасете не было больших численных значений
dataset['Цена, млн'] = dataset['Цена'] / 1_000_000
dataset.drop(columns=['Цена'], inplace=True)

In [304]:
# проверим на корректность (не должно быть отрицательных)
len(dataset[dataset['Цена, млн'] < 0])

0

In [305]:
# и проверим изменения
dataset['Цена, млн'].value_counts()

Цена, млн
6.100000    328
7.100000    309
6.200000    272
7.300000    272
6.000000    196
           ... 
8.345700      1
8.373800      1
8.401900      1
8.458100      1
5.733626      1
Name: count, Length: 4859, dtype: int64

### Очищенная цена за м², млн

In [306]:
# приведем к млн и другой валютный показатель
dataset['Очищенная цена за м², млн'] = dataset['Очищенная цена за м²'] / 1000

dataset.drop(columns=['Очищенная цена за м²'], inplace=True)

# и сразу проверим неотрицательность
len(dataset[dataset['Очищенная цена за м², млн'] < 0])

0

### Комиссия агенту

In [307]:
dataset['Комиссия агенту'].unique()

array([nan, 'без комиссии', '3%', '2%', '1%', '100%'], dtype=object)

In [308]:
dataset['Комиссия агенту'].value_counts()

Комиссия агенту
без комиссии    5183
100%              17
2%                 5
3%                 1
1%                 1
Name: count, dtype: int64

In [309]:
# заменим пропуски
dataset.loc[dataset[dataset['Комиссия агенту'].isna()].index, 'Комиссия агенту'] = 'Неизвестно'

# уберем наблюдения с малой частотой
dataset = dataset[~dataset['Комиссия агенту'].isin(['1%', '2%', '3%'])]


In [310]:
dataset['Комиссия агенту'].value_counts()

Комиссия агенту
Неизвестно      7778
без комиссии    5183
100%              17
Name: count, dtype: int64

Видим что комиссии либо нет, либо она неизвестна. Наблюдения с комиссией крайне разрежены относительно всего признака, поэтому просто его удалим.

In [311]:
dataset.drop(columns=['Комиссия агенту'], inplace=True)

### Общая площадь

In [312]:
dataset['Общая площадь'].isna().sum()

0

In [313]:
dataset['Общая площадь'].sample(5)

4888    85.42 м²
8573       52 м²
2834       43 м²
7032       42 м²
4770     66.4 м²
Name: Общая площадь, dtype: object

In [314]:
# выделим число
def extract_number(string) -> float | None:
    match = re.search(r'\d+(\.\d+)?', string)
    if match:
        return float(match.group())
    else:
        return None
    
dataset['Общая площадь'] = dataset['Общая площадь'].astype('str').apply(extract_number)

In [315]:
# проверка
dataset['Общая площадь'].sample(5)

11470    117.00
5853      86.41
2681      47.00
9660      30.00
5252      33.40
Name: Общая площадь, dtype: float64

In [316]:
dataset['Общая площадь'].isna().sum()

0

In [317]:
len(dataset[dataset['Общая площадь'] < 0])

0

### Возможна ипотека

In [318]:
dataset['Возможна ипотека'].unique()

array(['да', nan], dtype=object)

Замена этого nan по хорошему требует какого-то исследования, но т.к. сейчас не стоит задачи построить модель - заменим на "нет" (позднее увидим что таких значений будет менее 12%)

In [319]:
dataset.loc[dataset[dataset['Возможна ипотека'].isna()].index, 'Возможна ипотека'] = 'нет'

In [320]:
# проверим изменения
dataset['Возможна ипотека'].value_counts()

Возможна ипотека
да     11351
нет     1627
Name: count, dtype: int64

### Лифт

In [321]:
dataset['Лифт'].unique()

array([nan, 'да'], dtype=object)

In [323]:
# аналогично ипотеке заменим пропуски на 'нет'
dataset.loc[dataset[dataset['Лифт'].isna()].index, 'Лифт'] = 'нет'

In [324]:
# перепроверим
dataset['Лифт'].value_counts()

Лифт
да     9211
нет    3767
Name: count, dtype: int64

### Рейтинг школы

In [326]:
dataset['Рейтинг школы'].value_counts()

Рейтинг школы
Место в рейтинге №28     903
Место в рейтинге №120    753
Место в рейтинге №279    599
Место в рейтинге №603    418
Место в рейтинге №94     317
                        ... 
Место в рейтинге №99       1
Место в рейтинге №627      1
Место в рейтинге №601      1
Место в рейтинге №625      1
Место в рейтинге №537      1
Name: count, Length: 654, dtype: int64

In [327]:
dataset['Рейтинг школы'].isna().sum()

0

Для преобразования к категориальной шкале вычленим номер в рейтинге

In [328]:
dataset['Рейтинг школы'] = dataset['Рейтинг школы'].astype('str').apply(extract_number).astype('int64')

In [329]:
# перепроверка
dataset['Рейтинг школы'].value_counts()

Рейтинг школы
28     903
120    753
279    599
603    418
94     317
      ... 
99       1
627      1
601      1
625      1
537      1
Name: count, Length: 654, dtype: int64

In [331]:
len(dataset[dataset['Рейтинг школы'] <= 0])

0

### Расстояние до метро

In [332]:
dataset['Расстояние до метро'].isna().sum()

0

In [337]:
dataset['Расстояние до метро'].value_counts()

Расстояние до метро
995 м.                        742
37 мин,             7.1 км    733
1671 м.                       550
2 км                          371
36 мин,             8.5 км    365
                             ... 
1079 м.                         1
691 м.                          1
1689 м.                         1
29 мин,             2,4 км      1
58 мин,             4,8 км      1
Name: count, Length: 1812, dtype: int64

In [338]:
# приводим к метрам
to_replace = [r'\d+\s*мин,?\s*', r'\s*км\.?', r'\s*м\.?'] 
new_vals = ['', '000', '']

dataset['Расстояние до метро, м'] = pd.to_numeric(dataset['Расстояние до метро'].replace(to_replace, new_vals, regex=True), errors='coerce')

dataset.drop(columns=['Расстояние до метро'], inplace=True)

In [339]:
# перепроверка
dataset['Расстояние до метро, м'].isna().sum()

2171

In [341]:
dataset['Расстояние до метро, м'].sample(10)

2391      989.0
12499       7.1
10486     570.0
1624      704.0
10014     150.0
8550     1931.0
3726        NaN
11327    1244.0
3297      860.0
11961    3000.0
Name: Расстояние до метро, м, dtype: float64

### Расстояние до ближайшей школы

In [342]:
dataset['Расстояние до ближайшей школы'].isna().sum()

0

In [343]:
dataset['Расстояние до ближайшей школы'].value_counts()

Расстояние до ближайшей школы
~ 2.9 км       805
~ 1.4 км       609
570 м          443
~ 2.4 км       366
~ 1.2 км       334
              ... 
989 м            1
~ 8.5 км         1
37 м             1
~ 6.7 км         1
~ 2142.1 км      1
Name: count, Length: 1033, dtype: int64

In [344]:
# приводим к метрам
to_replace = [r'\s*км\.?', r'\s*м.?']
new_vals = ['000', '']

dataset['Расстояние до ближайшей школы, м'] = pd.to_numeric(dataset['Расстояние до ближайшей школы'].replace(to_replace, new_vals, regex=True), errors='coerce')

dataset.drop(columns=['Расстояние до ближайшей школы'], inplace=True)

In [345]:
# перепроверка
dataset['Расстояние до ближайшей школы, м'].isna().sum()

3129

In [346]:
dataset['Расстояние до ближайшей школы, м']

0        190.0
1        776.0
2        619.0
3        931.0
4        754.0
         ...  
12985      NaN
12986      NaN
12987      NaN
12988      NaN
12989      NaN
Name: Расстояние до ближайшей школы, м, Length: 12978, dtype: float64